In [1]:
import astropy
from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import WMAP9 as cosmo
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.image as mpimg
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import numpy as np
import gzip
from scipy.optimize import curve_fit
from scipy import stats
from scipy import odr
import math
import pymc3 as pm
import theano

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\nikko\anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pabdatazcut = fits.open('pabdatazcut.fits')
sortedpab = sorted(pabdatazcut[1].data , key = lambda data: data['PAB_FLUX'] , reverse = True )

data_dict= dict()
for obj in sortedpab:
    for key in ['FIELD', 'ID', 'RA', 'DEC' , 'Z_50', 'Z_84','Z_16' , 'PAB_FLUX', 'PAB_FLUX_ERR']:
        data_dict.setdefault(key, list()).append(obj[key])

sortedpabdf = pd.DataFrame(data_dict)
sortedpabdf['Z_ERR']= ((sortedpabdf['Z_84'] - sortedpabdf['Z_50']) + (sortedpabdf['Z_50'] - sortedpabdf['Z_16'])) / (2 * sortedpabdf['Z_50'])
sortedpabdf['OBS_PAB'] = 12820 * (1 + sortedpabdf['Z_50'])
sortedpabdf.loc[sortedpabdf['FIELD'] == "ERS" , 'FIELD'] = "ERSPRIME"
sortedpabdf = sortedpabdf[['FIELD' , 'ID' , 'RA' , 'DEC' , 'Z_50' , 'Z_ERR' , 'PAB_FLUX' , 'PAB_FLUX_ERR' , 'OBS_PAB']]

In [3]:
good = [6 , 9 , 12 , 14 , 15 , 18 , 21 , 24 , 29 , 30 , 35 , 36 , 37 , 46 , 48 , 51 , 52 , 55 , 58 , 70 , 71 , 72 , 74 , 75 , 78 , 80 , 81 , 84 , 89 , 97 , 98 , 102 ,  112 , 118 , 119 , 120 , 125 , 126 , 128 , 129 , 132 , 138 , 147 , 148 , 150 , 158 , 159 , 160 , 163 , 166 , 181 , 184 , 186 , 189 , 193 , 194 , 196 , 206 , 213 , 215 , 235 , 244 , 257 , 259 , 265 , 275 , 280 , 284 , 285 , 290 , 306 , 319 , 326 , 339 , 347 , 365 ]
gooddf50 = sortedpabdf.loc[good]

GNdf = gooddf50[gooddf50['FIELD'] == 'GN1']
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN2'])
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN3'])
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN4'])
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN5'])
GNdf = GNdf.append(gooddf50[gooddf50['FIELD'] == 'GN7'])
GNdf = GNdf[GNdf.ID != 38114]
GNdf = GNdf[GNdf.ID != 21829]
GNdf = GNdf[GNdf.ID != 22158]
GNdf = GNdf[GNdf.ID != 25434]
GNdf = GNdf[GNdf.ID != 35356]
GNdf = GNdf[GNdf.ID != 19075]
GNdf = GNdf[GNdf.ID != 22583]
GNdf = GNdf[GNdf.ID != 22933]
GNdf = GNdf[GNdf.ID != 26542]
GNdf = GNdf[GNdf.ID != 33015]
GNdf = GNdf[GNdf.ID != 37721]
GNdf = GNdf[GNdf.ID != 32992]
GNdf = GNdf[GNdf.ID != 36624]

GSdf = gooddf50[gooddf50['FIELD'] == 'GS1']
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'GS2'])
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'GS3'])
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'GS4'])
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'GS5'])
GSdf = GSdf.append(gooddf50[gooddf50['FIELD'] == 'ERSPRIME'])
GSdf = GSdf[GSdf.ID != 39505]
GSdf = GSdf[GSdf.ID != 41397]
GSdf = GSdf[GSdf.ID != 43823]
GSdf = GSdf[GSdf.ID != 43401]
GSdf = GSdf[GSdf.ID != 36971]
GSdf = GSdf[GSdf.ID != 26915]
GSdf = GSdf[GSdf.ID != 37739]
GSdf = GSdf[GSdf.ID != 36001]
GSdf = GSdf[GSdf.ID != 42866]
GSdf = GSdf[GSdf.ID != 39042]
GSdf = GSdf[GSdf.ID != 44831]
GSdf = GSdf[GSdf.ID != 44959]
GSdf = GSdf[GSdf.ID != 34363]
GSdf = GSdf[GSdf.ID != 36624]
GSdf = GSdf[GSdf.ID != 20623]
GSdf = GSdf[GSdf.ID != 24491]
GSdf = GSdf[GSdf.ID != 43700]

fulldf = pd.concat([GNdf , GSdf])
fulldf['DL'] = cosmo.luminosity_distance(fulldf['Z_50'])

#display(fulldf)
#print(len(fulldf))


In [4]:
GNtable = Table.read('D:/GOODSN.hdf5', path = 'data') 
GStable = Table.read('D:/GOODSS.hdf5', path = 'data')

data_dictn= dict()
for obj in GNtable:
    for key in ['id3DHST' , 'RA', 'DEC', 'ir_SFR-ladder_total' , 'ir_SFR-ladder_total_Av' ,'gf_n_j', 'gf_dn_j' , 'gf_f_j' , 'gf_re_j' , 'gf_dre_j' , 'td_Av', 'ACS_F435W_FLUX' , 'ACS_F775W_FLUX' , 'restalt_restUjohnson' , 'restalt_restVjohnson'  , 'td_flag1' , 'td_z_best' , 'td_z_spec' , 'td_z_best_s' , 'td_z_peak_phot' ,'td_lmass', 'td_sfr' , 'td_sfr_IR' , 'td_sfr_UV', 'ir_z_ir' ,'ir_SFR-UV_corr' , 'ir_SFR-UV_corr_Error','ir_SFR-IR', 'ir_UV_beta']:
        data_dictn.setdefault(key, list()).append(obj[key])

GNtabledf = pd.DataFrame(data_dictn)
GNtabledf = GNtabledf[GNtabledf['td_z_best'] <= .31]
GNtabledf = GNtabledf[GNtabledf['td_z_best'] >= .1]
data_dicts= dict()
for obj in GStable:
    for key in ['id3DHST' , 'RA', 'DEC', 'ir_SFR-ladder_total' , 'ir_SFR-ladder_total_Av' , 'gf_n_j', 'gf_dn_j' , 'gf_f_j' , 'gf_re_j' , 'gf_dre_j' , 'td_Av' , 'ACS_F435W_FLUX' , 'ACS_F775W_FLUX' , 'restUjohnson' , 'restVjohnson' , 'td_flag1' , 'td_z_best' , 'td_z_spec', 'td_z_best_s', 'td_z_peak_phot', 'td_lmass' , 'td_sfr' , 'td_sfr_IR' , 'td_sfr_UV' , 'ir_z_ir' , 'ir_SFR-UV_corr', 'ir_SFR-UV_corr_Error', 'ir_SFR-IR', 'ir_UV_beta']:
        data_dicts.setdefault(key, list()).append(obj[key])

GStabledf = pd.DataFrame(data_dicts)
GStabledf = GStabledf[GStabledf['td_z_best'] <= .31]

#display(GStabledf)
#display(GNtabledf)

In [7]:
idmergedf = pd.merge(GNdf, GNtabledf , how = 'inner' , left_on = 'ID' , right_on = 'id3DHST')
idmergedf.drop('RA_y', axis=1, inplace=True)
idmergedf.drop('DEC_y', axis=1, inplace=True)
idmergedf.loc[5 , 'Z_50'] = 0.21129
idmergedf.loc[5 , 'PAB_FLUX'] = 27.5076294
idmergedf.loc[5 , 'PAB_FLUX_ERR'] = 2.86222911
idmergedf['DL'] = cosmo.luminosity_distance(idmergedf['Z_50'])
idmergedf['PAB_L'] = idmergedf['PAB_FLUX'] * 4 * np.pi * idmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)
idmergedf['PAB_L_ERR'] = idmergedf['PAB_FLUX_ERR'] * 4 * np.pi * idmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)

idmergedfs = pd.merge(GSdf, GStabledf , how = 'inner' , left_on = 'ID' , right_on = 'id3DHST')
idmergedfs.drop('RA_y', axis=1, inplace=True)
idmergedfs.drop('DEC_y', axis=1, inplace=True)
idmergedfs['DL'] = cosmo.luminosity_distance(idmergedfs['Z_50'])
idmergedfs['PAB_L_ERR'] = idmergedfs['PAB_FLUX_ERR'] * 4 * np.pi * idmergedfs['DL']**2 * 9.521 * 10**(48) * 10**(-17)
idmergedfs['PAB_L'] = idmergedfs['PAB_FLUX'] * 4 * np.pi * idmergedfs['DL']**2 * 9.521 * 10**(48) * 10**(-17)
idmergedfs['restalt_restUjohnson'] = idmergedfs['restUjohnson'] 
idmergedfs['restalt_restVjohnson'] = idmergedfs['restVjohnson']

fullmergedf = pd.concat([idmergedf , idmergedfs] , sort = False)
fullmergedf.drop('restUjohnson', axis=1, inplace=True)
fullmergedf.drop('restVjohnson', axis=1, inplace=True)

fullmergedf['PAB_SFR'] = np.log10(fullmergedf['PAB_L'].astype(float)) - 40.02
fullmergedf['PAB_SFR_ERR']= .434*fullmergedf['PAB_L_ERR']/fullmergedf['PAB_L']
fullmergedf['UV_SFR_ERR']= .434*fullmergedf['ir_SFR-UV_corr_Error']/fullmergedf['ir_SFR-UV_corr']
#fullmergedf['PAB_SFR_EX'] = fullmergedf['PAB_SFR'] - fullmergedf['ir_SFR-UV_corr'] 
fullmergedf['PAB_SFR_EX2'] = fullmergedf['PAB_SFR'] - np.log10(fullmergedf['ir_SFR-UV_corr'])
fullmergedf['PAB_SFR_EX2_ERR'] = np.sqrt(fullmergedf['PAB_SFR_ERR'].astype(None)**2 + fullmergedf['UV_SFR_ERR'].astype(None)**2)
fullmergedf['PAB_SFR_LL'] = np.log10((min(fullmergedf['PAB_FLUX']) * 4 * np.pi * fullmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)).astype(float)) - 40.02
fullmergedf['PAB_SFR_EX2_LL'] = fullmergedf['PAB_SFR_LL'] - np.log10(fullmergedf['ir_SFR-UV_corr'])
fullmergedf['PAB_SFR_LL_1SIGMA'] = np.log10((min(fullmergedf['PAB_FLUX_ERR']) * 4 * np.pi * fullmergedf['DL']**2 * 9.521 * 10**(48) * 10**(-17)).astype(float)) - 40.02
fullmergedf['PAB_SFR_EX_LL_1SIGMA'] = fullmergedf['PAB_SFR_LL_1SIGMA'] - np.log10(fullmergedf['ir_SFR-UV_corr']) 
x = np.genfromtxt('D:/best-fit_sigma1.dat', dtype = None)
sigma1df = pd.DataFrame(x)
fullmergedf['SIGMA_1'] = sigma1df['f1']
display(fullmergedf)

,FIELD,ID,RA_x,DEC_x,Z_50,Z_ERR,PAB_FLUX,PAB_FLUX_ERR,OBS_PAB,id3DHST,...,PAB_SFR,PAB_SFR_ERR,UV_SFR_ERR,PAB_SFR_EX2,PAB_SFR_EX2_ERR,PAB_SFR_LL,PAB_SFR_EX2_LL,PAB_SFR_LL_1SIGMA,PAB_SFR_EX_LL_1SIGMA,SIGMA_1
0,GN1,37683,189.306089,62.360350,0.277481,0.001144,3.907847,1.367054,16377.307364,37683,...,-0.035078,0.151823,0.074554,-0.301065,0.169141,-1.127231,-1.393218,-1.315057,-1.581044,7.115943
1,GN2,19221,189.201266,62.240702,0.139631,0.002470,19.445505,2.126779,14610.064284,19221,...,-0.005105,0.0474671,0.051236,-0.279978,0.069845,-1.794139,-2.069013,-1.981966,-2.256839,8.092881
2,GN2,15610,189.212722,62.222423,0.199242,0.007588,14.520550,2.317139,15374.278308,15610,...,0.208897,0.0692562,0.018728,-0.006275,0.071744,-1.453302,-1.668474,-1.641128,-1.856300,7.696213
3,GN2,18157,189.182293,62.232458,0.196814,0.027064,3.889960,1.627647,15343.154062,18157,...,-0.375045,0.181595,0.067536,-0.164748,0.193747,-1.465206,-1.254908,-1.653032,-1.442735,7.376508
4,GN2,21693,189.232524,62.248474,0.281981,0.023462,2.519289,1.432714,16434.994103,21693,...,-0.209675,0.246815,0.031651,1.412949,0.248836,-1.111169,0.511456,-1.298995,0.323629,7.328494
5,GN3,34456,189.339813,62.324291,0.211290,0.010113,27.507629,2.862229,15695.781592,34456,...,0.543534,0.0451587,0.041423,0.234438,0.061279,-1.396135,-1.705231,-1.583962,-1.893057,8.551415
6,GN3,34157,189.206826,62.321196,0.245119,0.014142,28.282754,2.688524,15962.430241,34157,...,0.701403,0.0412555,0.035539,0.678385,0.054452,-1.250334,-1.273352,-1.438161,-1.461178,7.709024
7,GN3,33397,189.175470,62.314347,0.245053,0.010841,6.159068,1.908827,15961.576268,33397,...,0.039128,0.134506,0.035696,0.060551,0.139162,-1.250603,-1.229179,-1.438429,-1.417006,7.683212
8,GN3,33511,189.234550,62.314768,0.256401,0.009254,4.371808,1.187729,16107.064381,33511,...,-0.064933,0.117909,0.052148,0.443706,0.128926,-1.205809,-0.697171,-1.393636,-0.884998,7.320043
9,GN3,34368,189.338526,62.320968,0.228424,0.005207,4.132371,0.995737,15748.400334,34368,...,-0.203389,0.104577,0.010805,0.135720,0.105133,-1.319804,-0.980695,-1.507630,-1.168521,7.594131


In [7]:
def f(B, x):
    '''Linear function y = m*x + b'''
    # B is a vector of the parameters.
    # x is an array of the current x values.
    # x is in the same format as the x passed to Data or RealData.
    #
    # Return an array in the same format as y passed to Data or RealData.
    return B[0]*x + B[1]

def f0(B, x):
    '''Linear function y = m*x + b'''
    # B is a vector of the parameters.
    # x is an array of the current x values.
    # x is in the same format as the x passed to Data or RealData.
    #
    # Return an array in the same format as y passed to Data or RealData.
    return B[0]*x*0 + B[1]

def fpos(B , x):
    '''Linear function y = m*x + b'''
    # B is a vector of the parameters.
    # x is an array of the current x values.
    # x is in the same format as the x passed to Data or RealData.
    #
    # Return an array in the same format as y passed to Data or RealData.
    return B[0]*x + 0.29319574 

In [25]:
linear = odr.Model(f)

linear0 = odr.Model(f0)
linearpos = odr.Model(fpos)
linmass = odr.Model(f)

print('One Line')
mydata = odr.RealData(fullmergedf['ir_UV_beta'] , fullmergedf['PAB_SFR_EX2'] , sy = fullmergedf['PAB_SFR_EX2_ERR'])
myodr = odr.ODR(mydata, linear, beta0=[1., 2.])
myoutput = myodr.run()
myoutput.pprint()

print('\nTwo Line negative')
betaneg= odr.RealData(fullmergedf['ir_UV_beta'][fullmergedf['ir_UV_beta'] < 0] , fullmergedf['PAB_SFR_EX2'][fullmergedf['ir_UV_beta'] < 0] , sy = fullmergedf['PAB_SFR_EX2_ERR'][fullmergedf['ir_UV_beta'] < 0])
betanegodr = odr.ODR(betaneg, linear0 , beta0 = [0. , 0.])
betanegoutput = betanegodr.run()
betanegoutput.pprint()

print('\nTwo Line positive')
betapos= odr.RealData(fullmergedf['ir_UV_beta'][fullmergedf['ir_UV_beta'] > 0] , fullmergedf['PAB_SFR_EX2'][fullmergedf['ir_UV_beta'] > 0] , sy = fullmergedf['PAB_SFR_EX2_ERR'][fullmergedf['ir_UV_beta'] > 0])
betaposodr = odr.ODR(betapos, linearpos, beta0=[0])
betaposoutput = betaposodr.run()
betaposoutput.pprint()

print('\nTwo Line positive')
betapos= odr.RealData(fullmergedf['ir_UV_beta'][fullmergedf['ir_UV_beta'] > 0] , fullmergedf['PAB_SFR_EX2'][fullmergedf['ir_UV_beta'] > 0] , sy = fullmergedf['PAB_SFR_EX2_ERR'][fullmergedf['ir_UV_beta'] > 0])
betaposodr = odr.ODR(betapos, linear, beta0=[0 , 0])
betaposoutput = betaposodr.run()
betaposoutput.pprint()

print('\nMass fit beta < 0')
mass = odr.RealData(fullmergedf['td_lmass'][fullmergedf['ir_UV_beta'] < 0] , fullmergedf['PAB_SFR_EX2'][fullmergedf['ir_UV_beta'] < 0] , sy = fullmergedf['PAB_SFR_EX2_ERR'][fullmergedf['ir_UV_beta'] < 0] , sx = .2)
massodr = odr.ODR(mass, linmass, beta0=[0., 0.])
massoutput = massodr.run()
massoutput.pprint()  

One Line
Beta: [0.56344688 0.88720002]
Beta Std Error: [0.07880763 0.1133931 ]
Beta Covariance: [[0.00591861 0.00412532]
 [0.00412532 0.0122534 ]]
Residual Variance: 1.0493412247441196
Inverse Condition #: 0.5483223343358961
Reason(s) for Halting:
  Sum of squares convergence

Two Line negative
Beta: [0.         0.29319574]
Beta Std Error: [0.        0.0588609]
Beta Covariance: [[0.         0.        ]
 [0.         0.00039389]]
Residual Variance: 8.795971809675569
Inverse Condition #: 1.0
Reason(s) for Halting:
  Problem is not full rank at solution
  Both sum of squares and parameter convergence

Two Line positive
Beta: [0.56961082]
Beta Std Error: [0.1022489]
Beta Covariance: [[0.0085706]]
Residual Variance: 1.2198490771573813
Inverse Condition #: 1.0
Reason(s) for Halting:
  Sum of squares convergence

Two Line positive
Beta: [0.5430107  0.38003299]
Beta Std Error: [0.1873268  0.51082526]
Beta Covariance: [[ 0.02409073 -0.05357801]
 [-0.05357801  0.17914094]]
Residual Variance: 1.45

In [23]:
print('\nSersic fit')
sersic = odr.RealData(fullmergedf['gf_n_j'][fullmergedf['gf_f_j'] == 0] , fullmergedf['PAB_SFR_EX2'][fullmergedf['gf_f_j'] == 0] , sy = fullmergedf['PAB_SFR_EX2_ERR'][fullmergedf['gf_f_j'] == 0] , sx = fullmergedf['gf_dn_j'][fullmergedf['gf_f_j'] == 0] )
sersicodr = odr.ODR(sersic, linear, beta0=[0., 0.])
sersicoutput = sersicodr.run()
sersicoutput.pprint()

print('\nSigma 1 fit')
sigma = odr.RealData(fullmergedf['SIGMA_1'][fullmergedf['gf_f_j'] == 0] , fullmergedf['PAB_SFR_EX2'][fullmergedf['gf_f_j'] == 0] , sy = fullmergedf['PAB_SFR_EX2_ERR'][fullmergedf['gf_f_j'] == 0], sx = .3 )
sigmaodr = odr.ODR(sigma, linear, beta0=[0., 0.])
sigmaoutput = sigmaodr.run()
sigmaoutput.pprint()

print('\nEffective Radius fit')
re = odr.RealData(fullmergedf['gf_re_j'][fullmergedf['gf_f_j'] == 0] , fullmergedf['PAB_SFR_EX2'][fullmergedf['gf_f_j'] == 0] , sy = fullmergedf['PAB_SFR_EX2_ERR'][fullmergedf['gf_f_j'] == 0] , sx = fullmergedf['gf_dre_j'][fullmergedf['gf_f_j'] == 0] )
reodr = odr.ODR(re, linear, beta0=[0., 0.])
reoutput = reodr.run()
reoutput.pprint()


Sersic fit
Beta: [-0.43273994  0.71699238]
Beta Std Error: [0.13951913 0.16138485]
Beta Covariance: [[ 0.00285953 -0.00304992]
 [-0.00304992  0.00382607]]
Residual Variance: 6.807271596800986
Inverse Condition #: 0.18859455984950155
Reason(s) for Halting:
  Sum of squares convergence

Sigma 1 fit
Beta: [-49.67442797 371.4712911 ]
Beta Std Error: [ 322.66509474 2410.39590332]
Beta Covariance: [[  48151.37108326 -359703.3708339 ]
 [-359703.3708339  2687085.25209286]]
Residual Variance: 2.162197275364933
Inverse Condition #: 0.0001868914521965199
Reason(s) for Halting:
  Iteration limit reached

Effective Radius fit
Beta: [0.06881018 0.24422024]
Beta Std Error: [0.14627936 0.12836353]
Beta Covariance: [[ 0.00220172 -0.00170078]
 [-0.00170078  0.00169543]]
Residual Variance: 9.718589689643448
Inverse Condition #: 0.23123082736757655
Reason(s) for Halting:
  Sum of squares convergence


In [20]:
massfitdf = fullmergedf[['td_lmass' , 'PAB_SFR_EX2' , 'PAB_SFR_EX2_ERR']][fullmergedf['ir_UV_beta'] < 0]
massfitdf['MASS_ERR'] = .3
massfitdf = massfitdf[['td_lmass' , 'PAB_SFR_EX2' , 'MASS_ERR' , 'PAB_SFR_EX2_ERR' ]]
massfitdf.to_csv('massfit.csv')

betafitnegdf = fullmergedf[['ir_UV_beta' , 'PAB_SFR_EX2' , 'PAB_SFR_EX2_ERR']][fullmergedf['ir_UV_beta'] < 0]
betafitnegdf['BETA_ERR'] = .3
betafitnegdf = betafitnegdf[['ir_UV_beta' , 'PAB_SFR_EX2' , 'BETA_ERR' , 'PAB_SFR_EX2_ERR' ]]
betafitnegdf.to_csv('betafitneg.csv')

betafitposdf = fullmergedf[['ir_UV_beta' , 'PAB_SFR_EX2' , 'PAB_SFR_EX2_ERR']][fullmergedf['ir_UV_beta'] > 0]
betafitposdf['BETA_ERR'] = .3
betafitposdf = betafitnegdf[['ir_UV_beta' , 'PAB_SFR_EX2' , 'BETA_ERR' , 'PAB_SFR_EX2_ERR' ]]
betafitposdf.to_csv('betafitpos.csv')

betafitoldf = fullmergedf[['ir_UV_beta' , 'PAB_SFR_EX2' , 'PAB_SFR_EX2_ERR']]
betafitoldf['BETA_ERR'] = .3
betafitoldf = betafitnegdf[['ir_UV_beta' , 'PAB_SFR_EX2' , 'BETA_ERR' , 'PAB_SFR_EX2_ERR' ]]
betafitoldf.to_csv('betafitoneline.csv')

refitdf = fullmergedf[['gf_re_j' , 'PAB_SFR_EX2' , 'gf_dre_j' ,'PAB_SFR_EX2_ERR']][fullmergedf['gf_f_j'] == 0]
refitdf.to_csv('refit.csv')

sersicfitdf = fullmergedf[['gf_n_j' , 'PAB_SFR_EX2' , 'gf_dn_j' ,'PAB_SFR_EX2_ERR']][fullmergedf['gf_f_j'] == 0]
sersicfitdf.to_csv('sersicfit.csv')

sigmafitdf = fullmergedf[['SIGMA_1' , 'PAB_SFR_EX2' , 'PAB_SFR_EX2_ERR']][fullmergedf['gf_f_j'] == 0]
sigmafitdf['SIGMA_ERR'] = .3
sigmafitdf = sigmafitdf[['SIGMA_1' , 'PAB_SFR_EX2' , 'SIGMA_ERR' , 'PAB_SFR_EX2_ERR' ]]
sigmafitdf.to_csv('sigmafit.csv')


C:\Users\nikko\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
chidf = fullmergedf
chidf['CHI_MASS'] = (chidf['PAB_SFR_EX2'] - 1.09164224*chidf['td_lmass'] + 9.54330567)**2/.3**2
#chidf['CHI_BETA_OL'] = (chidf['PAB_SFR_EX2'] - *chidf['ir_UV_beta'] + )**2/.3**2
#chidf['CHI_BETA_2LNEG'] = (chidf['PAB_SFR_EX2'] - *chidf['ir_UV_beta'] + )**2/.3**2

display(chidf)
print('chi squared for mass', sum(chidf['CHI_MASS'][chidf['ir_UV_beta'] < 0]))

,FIELD,ID,RA_x,DEC_x,Z_50,Z_ERR,PAB_FLUX,PAB_FLUX_ERR,OBS_PAB,id3DHST,...,PAB_SFR_ERR,UV_SFR_ERR,PAB_SFR_EX2,PAB_SFR_EX2_ERR,PAB_SFR_LL,PAB_SFR_EX2_LL,PAB_SFR_LL_1SIGMA,PAB_SFR_EX_LL_1SIGMA,SIGMA_1,CHI_MASS
0,GN1,37683,189.306089,62.360350,0.277481,0.001144,3.907847,1.367054,16377.307364,37683,...,0.151823,0.074554,-0.301065,0.169141,-1.127231,-1.393218,-1.315057,-1.581044,7.115943,0.273176
1,GN2,19221,189.201266,62.240702,0.139631,0.002470,19.445505,2.126779,14610.064284,19221,...,0.0474671,0.051236,-0.279978,0.069845,-1.794139,-2.069013,-1.981966,-2.256839,8.092881,4.216650
2,GN2,15610,189.212722,62.222423,0.199242,0.007588,14.520550,2.317139,15374.278308,15610,...,0.0692562,0.018728,-0.006275,0.071744,-1.453302,-1.668474,-1.641128,-1.856300,7.696213,6.369831
3,GN2,18157,189.182293,62.232458,0.196814,0.027064,3.889960,1.627647,15343.154062,18157,...,0.181595,0.067536,-0.164748,0.193747,-1.465206,-1.254908,-1.653032,-1.442735,7.376508,1.800574
4,GN2,21693,189.232524,62.248474,0.281981,0.023462,2.519289,1.432714,16434.994103,21693,...,0.246815,0.031651,1.412949,0.248836,-1.111169,0.511456,-1.298995,0.323629,7.328494,21.914422
5,GN3,34456,189.339813,62.324291,0.211290,0.010113,27.507629,2.862229,15695.781592,34456,...,0.0451587,0.041423,0.234438,0.061279,-1.396135,-1.705231,-1.583962,-1.893057,8.551415,14.684098
6,GN3,34157,189.206826,62.321196,0.245119,0.014142,28.282754,2.688524,15962.430241,34157,...,0.0412555,0.035539,0.678385,0.054452,-1.250334,-1.273352,-1.438161,-1.461178,7.709024,0.236303
7,GN3,33397,189.175470,62.314347,0.245053,0.010841,6.159068,1.908827,15961.576268,33397,...,0.134506,0.035696,0.060551,0.139162,-1.250603,-1.229179,-1.438429,-1.417006,7.683212,3.895119
8,GN3,33511,189.234550,62.314768,0.256401,0.009254,4.371808,1.187729,16107.064381,33511,...,0.117909,0.052148,0.443706,0.128926,-1.205809,-0.697171,-1.393636,-0.884998,7.320043,3.561257
9,GN3,34368,189.338526,62.320968,0.228424,0.005207,4.132371,0.995737,15748.400334,34368,...,0.104577,0.010805,0.135720,0.105133,-1.319804,-0.980695,-1.507630,-1.168521,7.594131,2.081443


 chi squared for mass 351.17797247703203
